In [16]:
import pandas as pd
df = pd.read_csv("../data/processed/telco_customer_churn_cleaned.csv") 

In [17]:
#1: Drop leakage columns
leak_cols = [
    "Churn Reason",          # Known only after churn happens
    "Churn Category",        # Same
    "Customer Status"        # Future info (e.g., 'Churned')
]

df = df.drop(columns=leak_cols, errors="ignore")
#2: Drop high-identifier column
df = df.drop(columns=["CustomerID"], errors="ignore")
df = df.drop(columns=["count"], errors="ignore")
df = df.drop(columns=["latitude"], errors="ignore")
df = df.drop(columns=["longitude"], errors="ignore")


#3: Encode target variable
df["churn"] = df["churn_label"].map({"Yes": 1, "No": 0})


#4: Binary Yes/No encoding for features
yes_no_cols = [
    "Senior Citizen", "Partner", "Dependents", "Phone Service",
    "Paperless Billing"
]
for col in yes_no_cols:
    if col in df.columns:
        df[col] = df[col].map({"Yes": 1, "No": 0})


#5: One-hot encode categorical variables
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [18]:
df.to_csv("../data/processed/telco_model_ready.csv", index=False)

df.head()

,zip_code,tenure_months,monthly_charges,total_charges,churn_value,churn_score,cltv,churn,customerid_0003-MKNFE,customerid_0004-TLHLJ,...,churn_reason_Lack of self-service on Website,churn_reason_Limited range of services,churn_reason_Long distance charges,churn_reason_Moved,churn_reason_Network reliability,churn_reason_Poor expertise of online support,churn_reason_Poor expertise of phone support,churn_reason_Price too high,churn_reason_Product dissatisfaction,churn_reason_Service dissatisfaction
0,90003,2,53.85,108.15,1,86,3239,1,False,False,...,False,False,False,False,False,False,False,False,False,False
1,90005,2,70.70,151.65,1,67,2701,1,False,False,...,False,False,False,True,False,False,False,False,False,False
2,90006,8,99.65,820.50,1,86,5372,1,False,False,...,False,False,False,True,False,False,False,False,False,False
3,90010,28,104.80,3046.05,1,84,5003,1,False,False,...,False,False,False,True,False,False,False,False,False,False
4,90015,49,103.70,5036.30,1,89,5340,1,False,False,...,False,False,False,False,False,False,False,False,False,False
